# ActBlue donor profile

In [1]:
import json
import numpy as np
import pandas as pd
import psycopg2

## Connect to the PostgreSQL database

In [2]:
with open("config.json") as f:
    conf = json.load(f)
conn = psycopg2.connect(**conf)
conn.autocommit = True

## What are we analyzing?

How many reports are in our database?

In [3]:
pd.read_sql("""
SELECT count(*)
FROM fec_filings;
""", con=conn)

,count
0,289511


Which ActBlue reports do we have?

In [4]:
actblue_summaries = pd.read_sql("""
SELECT *
FROM fec_pac_summaries
JOIN fec_amended_filings USING (filing_id)
WHERE filer_committee_id_number = 'C00401224';
""", con=conn)

actblue_summaries.sort_values(by='coverage_through_date', ascending=False)
actblue_summaries[['filing_id','form_type','committee_name','coverage_from_date','coverage_through_date','report_code']]

,filing_id,form_type,committee_name,coverage_from_date,coverage_through_date,report_code
0,903093,F3XN,ActBlue,2013-12-01,2013-12-31,YE
1,907993,F3XN,ActBlue,2014-01-01,2014-01-31,M2
2,912211,F3XN,ActBlue,2014-02-01,2014-02-28,M3
3,921401,F3XN,ActBlue,2014-03-01,2014-03-31,M4
4,926005,F3XN,ActBlue,2014-04-01,2014-04-30,M5
5,932286,F3XN,ActBlue,2014-05-01,2014-05-31,M6
6,942637,F3XN,ActBlue,2014-06-01,2014-06-30,M7
7,946367,F3XN,ActBlue,2014-07-01,2014-07-31,M8
8,952738,F3XN,ActBlue,2014-08-01,2014-08-31,M9
9,961472,F3XN,ActBlue,2014-09-01,2014-09-30,M10
